<a href="https://colab.research.google.com/github/EYOSIYAS7/Machine_Learning_Projects/blob/main/toxic_comment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
!pip install pandas matplotlib sklearn

In [ ]:
!pip install pandas

In [ ]:
!pip install sklearn

In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np


In [ ]:
path = "../toxic comment/jigsaw-toxic-comment-classification-challenge/train/train.csv"

In [ ]:
testDataPath = "../toxic comment/test.csv/test.csv"

In [ ]:
testData = pd.read_csv(testDataPath)

In [ ]:
df = pd.read_csv(path)

In [ ]:
df.head()

In [ ]:
df.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


In [ ]:
df.iloc[10411]["comment_text"]

"No, I'm not. Link was forwarded to me. Please don't make assumptions about me. WP:AGF."

In [ ]:
df[df.columns[1:2]]

,comment_text
0,Explanation\nWhy the edits made under my usern...
1,D'aww! He matches this background colour I'm s...
2,"Hey man, I'm really not trying to edit war. It..."
3,"""\nMore\nI can't make any real suggestions on ..."
4,"You, sir, are my hero. Any chance you remember..."
...,...
159566,""":::::And for the second time of asking, when ..."
159567,You should be ashamed of yourself \n\nThat is ...
159568,"Spitzer \n\nUmm, theres no actual article for ..."
159569,And it looks like it was actually you who put ...


# Data preprocessing


In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
x = df["comment_text"]
y = df[df.columns[2:]].values

In [ ]:
x

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

In [ ]:
Max_words = 200000

In [ ]:
vectorizer = TextVectorization(max_tokens = Max_words, output_sequence_length=1800,output_mode="int")

In [ ]:
vectorizer.adapt(x.values)


In [ ]:
vectorized_text_test = vectorizer(x.values)

In [ ]:
vectorized_text_test

<tf.Tensor: shape=(1000, 1800), dtype=int64, numpy=
array([[2664,  239, 1189, ...,    0,    0,    0],
       [  31, 1358,    2, ...,    0,    0,    0],
       [  86, 4350, 9703, ...,    0,    0,    0],
       ...,
       [  41,   10,   43, ...,    0,    0,    0],
       [ 194,   34,  110, ...,    0,    0,    0],
       [1474, 7363, 6258, ...,    0,    0,    0]], dtype=int64)>

In [ ]:
# mcshbap , map cache shuffle batch prefetch . effective way of creating datapiplines
test_dataset = tf.data.Dataset.from_tensor_slices((vectorized_text_test, y))
test_dataset = test_dataset.cache()
test_dataset = test_dataset.shuffle(160000)
test_dataset = test_dataset.batch(16)
test_dataset = test_dataset.prefetch(8)

In [ ]:
batchx, batchy = dataset.as_numpy_iterator().next()

In [ ]:
batchx.shape

(16, 1800)

In [ ]:
batchy.shape

(16, 6)

In [ ]:
train = test_dataset.take(int(len(test_dataset)*.7))
val = test_dataset.skip(int(len(test_dataset)*.7)).take(int(len(test_dataset)*.2))
test = test_dataset.skip(int(len(test_dataset)*.9)).take(int(len(test_dataset)*.1))

In [ ]:
train_batch = train.as_numpy_iterator().next()

In [ ]:
trianx, trainy = train.as_numpy_iterator().next()

In [ ]:
trianx.shape

(16, 1800)

# Create sequential model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional , Dense, Embedding

In [ ]:
Embedding??

In [ ]:
model = Sequential()

model.add(Embedding(Max_words+1, 32))
model.add(Bidirectional(LSTM(32, activation= "tanh")) )
model.add(Dense(128, activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(6, activation="sigmoid"))


In [ ]:
model.compile(loss="BinaryCrossentropy", optimizer ="Adam")

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [ ]:
model = tf.keras.models.load_model('../toxic comment/toxicity (1).h5')

# Making prediction

In [ ]:
input_text = vectorizer("!")

In [ ]:
np.expand_dims(input_text,0).shape

(1, 1800)

In [ ]:
result = model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 0s 124ms/step


In [ ]:
result

array([[0.96461105, 0.01926725, 0.00296893, 0.02969842, 0.09555612,
        0.8138339 ]], dtype=float32)

In [ ]:
(result >0.4).astype(int)

array([[1, 0, 0, 0, 0, 1]])

In [ ]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [ ]:
result

array([[0.9984725 , 0.38181385, 0.9791897 , 0.01591792, 0.9045521 ,
        0.11702833]], dtype=float32)

# Model evaluation

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
x ,y = test.as_numpy_iterator().next()

In [ ]:
x

array([[     7,     20,  26775, ...,      0,      0,      0],
       [     5,    667,     34, ...,      0,      0,      0],
       [    51,   1146,   7475, ...,      0,      0,      0],
       ...,
       [    21,    675,     13, ...,      0,      0,      0],
       [   185,     10,     52, ...,      0,      0,      0],
       [   576, 182429,   4300, ...,      0,      0,      0]], dtype=int64)

In [ ]:
for batch in test.as_numpy_iterator():

    X_true, y_true = batch

    y_hat = model.predict(X_true)
    y_hat = (y_hat >0.4).astype(int)

    y_true.flatten()
    y_hat.flatten()

    pre.update_state(y_true,y_hat)
    re.update_state(y_true,y_hat)
    acc.update_state(y_true,y_hat)

In [ ]:
print(f'Precision : {pre.result().numpy()},Recall : {re.result().numpy()},Accuracy : {acc.result().numpy()}')

Precision : 0.8866087794303894,Recall : 0.4464462995529175,Accuracy : 0.9942989349365234


In [ ]:
pre.result().numpy()

0.8134206

In [ ]:
labelPath  =  "../toxic comment/test_labels.csv/test_labels.csv"

In [ ]:
labels = pd.read_csv(labelPath)

In [ ]:
labels.replace(-1, 1)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,1,1,1,1,1,1
1,0000247867823ef7,1,1,1,1,1,1
2,00013b17ad220c46,1,1,1,1,1,1
3,00017563c3f7919a,1,1,1,1,1,1
4,00017695ad8997eb,1,1,1,1,1,1
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,1,1,1,1,1,1
153160,fffd7a9a6eb32c16,1,1,1,1,1,1
153161,fffda9e8d6fafa9e,1,1,1,1,1,1
153162,fffe8f1340a79fc2,1,1,1,1,1,1


In [ ]:
labels.columns[2:]

Index(['severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], dtype='object')

Model Inference

In [ ]:
input_text = vectorizer("You 're a stupid man, I 'm going to teach you about the ball, but they 're men who don 't know the ball, they 're stupid guys who despise the women.")

In [ ]:
result = model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 0s 112ms/step


In [ ]:
(result > 0.4).astype(int)

array([[1, 0, 0, 0, 1, 0]])

The corresponding values of the array

In [ ]:
last.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')